In [2]:
import sys
sys.path.append('../src')
from logger import log, debug
from cleaner import clean_corpus_basic
from reader import read_files
from svm_utils import get_best_tokens_dummy, transform_sentence, get_score

from sklearn.utils import shuffle
from sklearn import svm

import numpy as np
import matplotlib.pyplot as plt

import nltk
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lefunes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/lefunes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# -------------- Hiperparametros -----------------

train_to_test = 0.8

In [4]:
# Leemos los archivos
corpus = clean_corpus_basic(read_files("../dataset/", ["negative", "positive"]))

# Filtramos por longitud
debug("[El corpus tiene " + str(len(corpus)) + " rows]")

#corpus["length"] = corpus["content"].map(lambda x: len(x.split(" ")))
#corpus = corpus[(corpus["length"] >= min_lenght) & (corpus["length"] <= max_length)]

debug("[Luego de filtrar el corpus tiene " + str(len(corpus)) + " rows]")

#Shuffleamos el corpus
corpus = shuffle(corpus)
corpus.reset_index(inplace=True, drop=True)


[Leyendo archivos en panda...]
[Archivos Leidos...]
[Usando cleaner basico]
[Limpiando el corpus...]
[Usando 8 threads ...]
[El corpus tiene 20000 rows]
[Luego de filtrar el corpus tiene 20000 rows]


In [5]:
def supertrain(train_set):
    train_set_splitted = np.array_split(train_set, 5)
    max_classifier = (None, 0)

    # la idea es hacer un cross-check, con 5 arrays
    splitted_len = len(train_set_splitted)
    for i in range(splitted_len):
        no_better_count = 0
        for _ in range(20): #Hacemos 20 iteraciones
            if(no_better_count > 5): #Despues de 3 iteraciones sin encontrar mejor, paramos
                break
                
            train_subset = train_set_splitted[0:i] + train_set_splitted[(i + 1): splitted_len]
            train_subset = [item for sublist in train_subset for item in sublist]
            np.random.shuffle(train_subset)

            np.random.shuffle(train_set_splitted[i])
                
            # Creamos el clasificador y lo evaluamos
            unzipped = list(zip(*train_subset)) 
            classifier = svm.SVC(gamma='scale', probability=True)
            classifier.fit(unzipped[0], unzipped[1])
            acc = get_score(classifier, train_set_splitted[i])

            # Vemos si encontramos un clasificador mejor
            if(acc > max_classifier[1]):
                log("[   Se encontro un clasificador mejor " + str(acc * 100) + "% ]")
                max_classifier = (classifier, acc) # Guardamos el mejor clasificador
            no_better_count += 1 # Si no encontramos mejor, aumentamos en uno

    return max_classifier


In [6]:
results_svm = []

each_q = 4
vector_size = 3

log("[Entrenando con parametro " + str(each_q) + " " + str(vector_size) + "]")
best_tokens = get_best_tokens_dummy(corpus, each_q)
best_tokens.to_pickle("best_tokens_svm.bin")

corpus["vector"] = corpus["content"].map(lambda x: transform_sentence(x, best_tokens, vector_size))

# Dividimos el corpus

subset = corpus[['vector','rate']]
featuresets = [tuple(x) for x in subset.values]
train, test = np.split(featuresets, [int(train_to_test * len(featuresets))])
clasif = supertrain(train)

real_acc = get_score(clasif[0], test)

log("[   Accuaracy real " + str(real_acc * 100) + "% ]")

results_svm.append((each_q, vector_size, real_acc, clasif[0], clasif[1]))
log("[Fin del entrenamiento]")

f = open("svm-no-restricted-4-3.bin", "wb+")
pickler = pickle.Pickler(f)
pickler.dump(clasif[0])

f.close()

[Entrenando con parametro 4 3]
[   Se encontro un clasificador mejor 89.25% ]
[   Se encontro un clasificador mejor 89.40625% ]
[   Accuaracy real 88.9% ]
[Fin del entrenamiento]
